In [37]:
import os
import uuid
import mlflow
import requests
import pandas as pd
from io import BytesIO
import numpy as np



In [34]:
year = 2023
month = 4
taxi_type = 'green'

url = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
# input_file = f'https://s3.amazonaws.com/nyc-tlc/trip+data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet'
output_file = f'output/{taxi_type}'
file_name = f"{year:04d}-{month:02d}.parquet"

RUN_ID = 'f36a894b060442d28b492636581cad8c'

In [40]:
def generate_uuids(n):
    ride_ids = []
    for i in range(n):
        ride_ids.append(str(uuid.uuid4()))
    return ride_ids

def read_dataframe(url: str):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(response.text)
    df = pd.read_parquet(BytesIO(response.content))

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    df['ride_id'] = generate_uuids(len(df))

    return df


def prepare_dictionaries(df: pd.DataFrame):
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']

    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [41]:
def load_model(run_id):
    logged_model = f's3://ml-model-bucket/1/{run_id}/artifacts/model'
    model = mlflow.pyfunc.load_model(logged_model)
    return model


def apply_model(url, run_id, output_file, file_name):

    df = read_dataframe(url)
    dicts = prepare_dictionaries(df)

    
    model = load_model(run_id)
    y_pred = model.predict(dicts)

    df_result = pd.DataFrame()
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    # df_result['ride_id'] = df['ride_id']
    # df_result['lpep_pickup_datetime'] = df['lpep_pickup_datetime']
    # df_result['PULocationID'] = df['PULocationID']
    # df_result['DOLocationID'] = df['DOLocationID']
    # df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    # df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    # df_result['model_version'] = run_id
    
    if not os.path.exists(output_file):
        os.makedirs(output_file)
    path = os.path.join(output_file, file_name)
    # df_result.to_parquet(path, index=False)
    df_result.to_parquet(
    path,
    engine='pyarrow',
    compression=None,
    index=False)
    return y_pred


In [42]:
y_pred = apply_model(url=url, run_id=RUN_ID, output_file=output_file, file_name=file_name)

In [43]:
np.mean(y_pred)
# y_pred

14.010841406437127